# Analysis

In [1]:
import pandas as pd
import plotnine as pn

In [ ]:
# Load data

params = [
    0, 1, 2, 4, 8, 16, # nosofsky
    # 0.1, 0.2, 0.5, 0.6, 0.75, 1, 2, 3, 5, 1000 # exp
]

# Learned
# dynamic = "reinforcement_learning"
dynamic = "replicator_dynamic"

# num_trials = 10
num_trials = 100

# similarity = "nosofsky"
similarity = "nosofsky_normed"
# similarity = "exp"
# similarity = "exp_normed"

distortion = "squared_dist"
# distortion = "abs_dist"

out = "multirun"
# out = "outputs"

# trajectory = True
trajectory = False


#############################################################################
# Below three saved in outputs, not multirun

# Pareto frontier
curve_fn = f"/Users/nathanielimel/lps/projects/rdsg/outputs/states=10/signals=10/distortion={distortion}/curve_points.csv"

# Counterpart points
counterpart_fn = f"/Users/nathanielimel/lps/projects/rdsg/outputs/states=10/signals=10/distortion={distortion}/counterpart_points.csv"

# Explored hypothetical systems
sampled_fn = f"/Users/nathanielimel/lps/projects/rdsg/outputs/states=10/signals=10/distortion={distortion}/sampled_points.csv"

#############################################################################
# Typically saved in multirun
# Simulation points
param_fns = {
    f"{float(param)}":f"/Users/nathanielimel/lps/projects/rdsg/{out}/states=10/signals=10/distortion={distortion}/similarity={similarity}/dynamics={dynamic}/num_trials={num_trials}/sim_param={param}/simulation_points.csv"
    for param in params
}

variant_fns = {key: val.replace("simulation_points", "variants_points") for key, val in param_fns.items()}

if trajectory:
    param_fns = {key: val.replace("simulation_points", "mean_points") for key, val in param_fns.items()}



In [ ]:
# load dataframes
curve_data = pd.read_csv(curve_fn)
counterpart_data = pd.read_csv(counterpart_fn)
sampled_data = pd.read_csv(sampled_fn)

simulation_dataframes = {
    k: pd.read_csv(param_fns[k]) for k in param_fns
}

variant_dataframes = {
    k: pd.read_csv(variant_fns[k]) for k in variant_fns
}

In [ ]:
param_names = {"exp": "beta", "nosofsky": "alpha"}
for name in param_names:
    if name in similarity:
        param_name = param_names[name]

# combine simulation dataframes
for key in simulation_dataframes:
    df = simulation_dataframes[key]
    df[param_name] = key # use string for category

sim_data = pd.concat(simulation_dataframes.values())
sim_data

In [ ]:
# combine variant dataframes
# combine simulation dataframes
for key in variant_dataframes:
    df = variant_dataframes[key]
    df[param_name] = key # use string for category

variant_data = pd.concat(variant_dataframes.values())
variant_data

In [ ]:
# Optional: Cast to int if all params are ints

sim_data[param_name] = sim_data[param_name].astype(float)
sim_data[param_name] = sim_data[param_name].astype(int)
sim_data[param_name] = sim_data[param_name].astype(str)

variant_data[param_name] = variant_data[param_name].astype(float)
variant_data[param_name] = variant_data[param_name].astype(int)
variant_data[param_name] = variant_data[param_name].astype(str)

counterpart_data[param_name] = counterpart_data[param_name].astype(float)
counterpart_data[param_name] = counterpart_data[param_name].astype(int)
counterpart_data[param_name] = counterpart_data[param_name].astype(str)

sim_data

In [ ]:
# plot each language, with a unique color for each gamma

# hard to clean this up with pd.Categorical
if param_name == "alpha":
    counterpart_data = counterpart_data.assign(
            alpha=pd.Categorical(
                counterpart_data["alpha"], 
                categories=[str(param) for param in params]
            )
        )

    sim_data = sim_data.assign(
            alpha=pd.Categorical(
                sim_data["alpha"], 
                categories=[str(param) for param in params]
            )
        )
else:
    counterpart_data = counterpart_data.assign(
            beta=pd.Categorical(
                counterpart_data["beta"], 
                categories=[str(param) for param in params]
            )
        )

    sim_data = sim_data.assign(
            beta=pd.Categorical(
                sim_data["beta"], 
                categories=[str(param) for param in params]
            )
        )


# Language keys
sim_data["language"] = {"reinforcement_learning": "learned", "replicator_dynamic": "evolved"}[dynamic]
# counterpart_data["language"] = "optimal counterpart"
counterpart_data["language"] = "optimal \ncounterpart"
all_data = pd.concat([counterpart_data, sim_data])

sampled_data["language"] = "hypothetical"
variant_data["language"] = "hypothetical"

# Trajectory keys
if trajectory:
    sim_data["time step"] = sim_data["round"]

counterpart_data


In [ ]:
plot = (
    # Set data and the axes
    pn.ggplot(
        data=curve_data, mapping=pn.aes(x="rate", y="distortion")
    )  
    + pn.geom_point(  # sampled langs
        # sampled_data,
        variant_data,
        pn.aes(shape="language"),
        color="gray",
        size=5,
        alpha=0.2,
    )
    + pn.geom_line(size=1) # pareto data                   
    + pn.geom_jitter( # simulation langs
        data=sim_data,
        mapping=pn.aes(
            color=param_name, 
            shape="language",
            ),
        alpha=0.3,
        size=5,
        # height=0.1,
    )
    + pn.geom_point( # theoretical bound langs last
        data=counterpart_data,
        mapping=pn.aes(
            color=param_name,
            shape="language",
            ),
        size=5,
    )    
    + pn.xlab("Complexity $I(S;\hat{S})$")
    + pn.ylab("Communicative Cost $D[S, \hat{S}]$")
    + pn.theme_seaborn(context="paper")
    + pn.theme(text=pn.element_text(size=18))
)
print(plot)

In [ ]:
# Save plot (will overwrite!)

plot_save_dir = f"/Users/nathanielimel/lps/projects/rdsg/multirun/states=10/signals=10/distortion={distortion}/similarity={similarity}/dynamics={dynamic}/num_trials={num_trials}"
fn = plot_save_dir + f"/multiple_{param_name}.png"
plot.save(filename=fn, width=10, height=10, dpi=300)
print(fn)

## Plot trajectories

In [ ]:
# Load data
mean_traj_fns = {
    key: val.replace("simulation_points", "mean_points") for key, val in param_fns.items()
}

mean_traj_dataframes = {
    k: pd.read_csv(mean_traj_fns[k]) for k in mean_traj_fns
}

# combine simulation dataframes
for key in mean_traj_dataframes:
    df = mean_traj_dataframes[key]
    df["alpha"] = key # use string for category

mean_traj_data = pd.concat(mean_traj_dataframes.values())
mean_traj_data = mean_traj_data.dropna() # unnecessary if all files were trajectories

mean_traj_data

In [ ]:
mean_traj_data["alpha"] = mean_traj_data["alpha"].astype(float)
mean_traj_data["alpha"] = mean_traj_data["alpha"].astype(int)
mean_traj_data["alpha"] = mean_traj_data["alpha"].astype(str)

mean_traj_data = mean_traj_data.assign(
    alpha=pd.Categorical(
        mean_traj_data["alpha"],
        categories=[str(param) for param in params]
    )
)

mean_traj_data["language"] = {"reinforcement_learning": "learned", "replicator_dynamic": "evolved"}[dynamic]

mean_traj_data

In [ ]:
# Extract the final rounds

final_round = mean_traj_data["round"].max()
final_round_data = mean_traj_data[mean_traj_data["round"] == final_round]


In [ ]:
# try latex for color

# r'$\sin (x)$'

final_round_data["$\alpha$"] = final_round_data["alpha"]
mean_traj_data["$\alpha$"] = mean_traj_data["alpha"]

In [ ]:
plot = (
    # Set data and the axes
    pn.ggplot(
            data=curve_data, mapping=pn.aes(x="rate", y="distortion")
    )  # pareto data    
    + pn.geom_point(  # sampled langs
        # sampled_data,
        variant_data,
        pn.aes(shape="language"),
        color="gray",
        size=5,
        alpha=0.2,
    )
    + pn.geom_line(  # simulation langs
        data=mean_traj_data,
        mapping=pn.aes(color="alpha"),
        # shape="o",
        alpha=1.0,
        size=3,
    )
    + pn.geom_line(size=1) # pareto     
    + pn.geom_point(  # final langs
        data=final_round_data,
        mapping=pn.aes(color="alpha"),
        shape="X",
        alpha=1.0,
        size=5,
    )
    + pn.xlab("Complexity $I(S;\hat{S})$")
    + pn.ylab("Communicative Cost $D[S, \hat{S}]$")
    + pn.theme_seaborn(context="paper")
    + pn.theme(text=pn.element_text(size=18))
)
print(plot)

In [ ]:
# Save plot (will overwrite!)

plot_save_dir = f"/Users/nathanielimel/lps/projects/rdsg/multirun/states=10/signals=10/distortion={distortion}/similarity={similarity}/dynamics={dynamic}/num_trials={num_trials}"
fn = plot_save_dir + "/alpha_trajectories.png"
plot.save(filename=fn, width=10, height=10, dpi=300)

# Perform statistical analyses

### Measure optimality

In [ ]:
# N.B.: The reason we need to copypaste this function is because it is sometimes more convenient to measure optimality all at once here, instead of running python script in a hydra sweep (if I forgot to)
import numpy as np
from scipy.spatial.distance import cdist

def measure_optimality(data: pd.DataFrame, curve_data: pd.DataFrame) -> np.ndarray:
    """Compute the min distance to any point on the frontier, for every point. Requires `data` to contain more than one row."""
    # get curve points as list of pairs
    pareto_points = np.array(list(curve_data[["rate", "distortion"]].itertuples(index=False, name=None)))
    points = np.array(list(data[["rate", "distortion"]].itertuples(index=False, name=None)))
    # N.B.: do not interpolate, so you don't measure high-dist random langs as more optimal than they are!

    # Measure closeness of each language to any frontier point
    distances = cdist(points, pareto_points)
    min_distances = np.min(distances, axis=1)

    # max complexity will be achieved by B-A    
    max_complexity = pareto_points[:, 0].max()
    # points may have higher cost than pareto max cost    
    max_cost = max(points[:,1].max(), pareto_points[:,].max())
    # max possible distance is sqrt( max_rate^2 + (max_distortion)^2 )
    max_distance = np.sqrt(max_cost**2 + max_complexity**2)

    # just use the max found distance? 
    # max_distance = np.max(min_distances)

    min_distances /= max_distance
    optimalities = 1 - min_distances
    return optimalities
    
    # return min_distances

In [ ]:
sim_data["optimality"] = measure_optimality(sim_data, curve_data)
variant_data["optimality"] = measure_optimality(variant_data, curve_data)
# sampled_data["optimality"] = measure_optimality(sampled_data, curve_data)


In [ ]:
display(sim_data.mean())
display(variant_data.mean())


In [ ]:
display(sim_data.min())
display(variant_data.min())
display(sampled_data.min())

### Show optimality in violin plots
*N.B.: need to change python interpreter to 'seaborn'!*

In [ ]:
# N.B.: good violin plots requires seaborn 0.12.2, which is incompatible with plotnine
import seaborn as sns
import matplotlib.pyplot as plt
plt.ion()
sns.set_theme()

In [ ]:

data = pd.concat([sim_data, variant_data])

violin = sns.catplot(
    data=data, y="alpha", x="optimality", hue="language", kind="violin", inner=None, split=False, cut=0, bw=.15, palette="pastel"
)

# violin.set(xlim=(None, None))

# sns.catplot(
#     data=tips, x="day", y="total_bill", hue="sex",
#     kind="violin", inner="stick", split=True, palette="pastel",
# )

plt.show()
# plt.savefig("rep_violin.png")

### be messy and load up rl data

In [ ]:
# load up rl data by hand, not from saved

# Simulation points
param_fns_rl = {
    f"{float(param)}":f"/Users/nathanielimel/lps/projects/rdsg/{out}/states=10/signals=10/distortion={distortion}/similarity={similarity}/dynamics=reinforcement_learning/num_trials={num_trials}/sim_param={param}/simulation_points.csv"
    for param in params
}

variant_fns_rl = {key: val.replace("simulation_points", "variants_points") for key, val in param_fns.items()}


simulation_dataframes_rl = {
    k: pd.read_csv(param_fns_rl[k]) for k in param_fns_rl
}

variant_dataframes_rl = {
    k: pd.read_csv(variant_fns_rl[k]) for k in variant_fns_rl
}


# combine simulation dataframes
for key in simulation_dataframes_rl:
    df = simulation_dataframes_rl[key]
    df[param_name] = key # use string for category

sim_data_rl = pd.concat(simulation_dataframes_rl.values())
display(sim_data_rl)


# combine
for key in variant_dataframes_rl:
    df = variant_dataframes_rl[key]
    df[param_name] = key # use string for category

variant_data_rl = pd.concat(variant_dataframes_rl.values())
display(variant_data_rl)

In [ ]:
# more messiness

sim_data_rl["optimality"] = measure_optimality(sim_data_rl, curve_data)
variant_data_rl["optimality"] = measure_optimality(variant_data_rl, curve_data)


sim_data_rl[param_name] = sim_data_rl[param_name].astype(float)
sim_data_rl[param_name] = sim_data_rl[param_name].astype(int)
sim_data_rl[param_name] = sim_data_rl[param_name].astype(str)

variant_data_rl[param_name] = variant_data_rl[param_name].astype(float)
variant_data_rl[param_name] = variant_data_rl[param_name].astype(int)
variant_data_rl[param_name] = variant_data_rl[param_name].astype(str)


sim_data_rl = sim_data_rl.assign(
            alpha=pd.Categorical(
                sim_data_rl["alpha"], 
                categories=[str(param) for param in params]
            )
        )

variant_data_rl = variant_data_rl.assign(
            alpha=pd.Categorical(
                variant_data_rl["alpha"],
                categories=[str(param) for param in params]
            )
)


sim_data_rl["language"] = "learned"
variant_data_rl["language"] = "hypothetical"

sim_data_rl

In [ ]:
# get violin for both
rep_data = pd.concat([sim_data, variant_data])
rl_data = pd.concat([sim_data_rl, variant_data_rl])
dynamics_data = pd.concat([rep_data, rl_data])

# change language to categorical to customize ordering
dynamics_data = dynamics_data.assign(
        language=pd.Categorical(
                dynamics_data["language"], 
                categories=["learned", "evolved", "hypothetical"]
            )
        )

In [ ]:

violin = sns.catplot(
    data=dynamics_data,
    x="alpha", y="optimality", 
    hue="language", 
    kind="violin", 
    inner=None, 
    split=False, 
    cut=0, 
    bw=.05, 
    palette="pastel",
    scale="width",
)
plt.show()

In [ ]:
dynamics_data = dynamics_data.assign(
        alpha=pd.Categorical(
                dynamics_data["alpha"], 
                categories=[str(param) for param in params]
            )
        )

## Try plotnine for violins

In [ ]:
# want alpha to be floats again

values = {"learned": "orange", "evolved": "green", "hypothetical": "blue"}

plot = (
    # Set data and the axes
    pn.ggplot(
            data=dynamics_data, mapping=pn.aes(x="alpha", y="optimality")
    )
    + pn.geom_violin(
        data=dynamics_data,
        mapping=pn.aes(
            fill="language", 
            color="language", 
            ),
        scale="width",
        bw=0.0015,
    )
    + pn.scale_color_manual(values=values)
    + pn.scale_fill_manual(values=values)
    + pn.xlab("Perceptual imprecision ($\\alpha$)")
    + pn.ylab("Optimality (Pareto-closeness)")
    + pn.theme_seaborn(context='paper')
    + pn.theme(text=pn.element_text(size=18))
)
print(plot)

In [ ]:
# save plot
fn = "optimality.png"
plot.save(filename=fn, width=10, height=10, dpi=300)

### other junk

In [ ]:
# what does hypothetical look like?
data = pd.concat([sim_data, variant_data])

sns.displot(data=data, x="rate", y="deviation from optimality", hue="language",)

In [ ]:
# save data for messing around later with plots

sim_data.to_csv("rl.csv", index=False)
# sampled_data.to_csv("explored.csv")
variant_data.to_csv("rl_variants.csv", index=False)


# sim_data.to_csv("rep_dyn.csv", index=False)
# variant_data.to_csv("rep_dyn_variants.csv", index=False)

## Figure with subplots

In [ ]:
import patchworklib as pw
print(pw.__version__)
print(pn.__version__)

In [ ]:
plot1 = (
    # Set data and the axes
    pn.ggplot(
        data=curve_data, mapping=pn.aes(x="rate", y="distortion")
    )  
    + pn.geom_point(  # sampled langs
        # sampled_data,
        variant_data,
        pn.aes(shape="language"),
        color="gray",
        size=5,
        alpha=0.2,
    )
    + pn.geom_line(size=1) # pareto data                   
    + pn.geom_jitter( # simulation langs
        data=sim_data,
        mapping=pn.aes(
            color=param_name, 
            shape="language",
            ),
        alpha=0.3,
        size=5,
        # height=0.1,
    )
    + pn.geom_point( # theoretical bound langs last
        data=counterpart_data,
        mapping=pn.aes(
            color=param_name,
            shape="language",
            ),
        size=5,
    )    
    + pn.xlab("Complexity $I(S;\hat{S})$")
    + pn.ylab("Communicative Cost $D[S, \hat{S}]$")
    + pn.ggtitle("Roth-Erev Reinforcement Learning")
    # + pn.theme_seaborn(context="paper")
    # + pn.theme(text=pn.element_text(size=18))
    + pn.theme(
                legend_title=pn.element_blank(),
                figure_size=(12, 9),
                legend_position='none',
                legend_box='horizontal',
        )
)

In [ ]:
plot2 = (
    # Set data and the axes
    pn.ggplot(
        data=curve_data, mapping=pn.aes(x="rate", y="distortion")
    )  
    + pn.geom_point(  # sampled langs
        # sampled_data,
        variant_data_rl,
        pn.aes(shape="language"),
        color="gray",
        size=5,
        alpha=0.2,
    )
    + pn.geom_line(size=1) # pareto data                   
    + pn.geom_jitter( # simulation langs
        data=sim_data,
        mapping=pn.aes(
            color=param_name, 
            shape="language",
            ),
        alpha=0.3,
        size=5,
        # height=0.1,
    )
    + pn.geom_point( # theoretical bound langs last
        data=counterpart_data,
        mapping=pn.aes(
            color=param_name,
            shape="language",
            ),
        size=5,
    )    
    + pn.xlab("Complexity $I(S;\hat{S})$")
    + pn.ylab("Communicative Cost $D[S, \hat{S}]$")
    + pn.ggtitle("Replicator Dynamic")
    # + pn.theme_seaborn(context="paper")
    # + pn.theme(text=pn.element_text(size=18))
    + pn.theme(
                legend_title=pn.element_blank(),
                figure_size=(12, 9),
                legend_position='right',
                legend_box='horizontal',
                legend_text=pn.element_text(size=18)
        )

)

In [ ]:
# combine figs

plot1 = pw.load_ggplot(plot1, figsize=(4,3))
plot2 = pw.load_ggplot(plot2, figsize=(4,3))

g = plot1 | plot2
g.savefig("example.png")